In [ ]:
import requests
import json
import csv
import os
import zipfile
import pandas as pd
from datetime import datetime, timedelta
import time
import math
import random

class TTCDataTransformer:
    def __init__(self):
        self.gtfs_package_id = "b811ead4-6eaf-4adb-8408-d389fb5a069c"
        self.delay_package_id = "e271cdae-8788-4980-96ce-6a5c95bc6618"
        self.base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action"
        
        # Paths - fixed for Jupyter compatibility
        try:
            # This works when running as a script
            self.script_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            # This works in Jupyter notebooks
            self.script_dir = os.getcwd()
        
        self.input_data_folder = os.path.join(self.script_dir, "input_data")
        self.output_data_folder = os.path.join(self.script_dir, "assets", "data")
        
        # Create folders
        self.ensure_folder_exists(self.input_data_folder)
        self.ensure_folder_exists(self.output_data_folder)
        
        # Session for requests
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'TTC-Data-Transformer/1.0'
        })

    def ensure_folder_exists(self, folder_path):
        """Create folder if it doesn't exist"""
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"📁 Created folder: {folder_path}")

    def fetch_package(self, package_id):
        """Fetch package information from CKAN API"""
        url = f"{self.base_url}/package_show?id={package_id}"
        response = self.session.get(url)
        response.raise_for_status()
        data = response.json()
        
        if not data.get('success'):
            raise Exception(f"API request failed: {data.get('error', {}).get('message', 'Unknown error')}")
        
        return data['result']

    def fetch_datastore_data(self, resource_id, limit=50000):
        """Fetch data from datastore resource"""
        url = f"{self.base_url}/datastore_search?id={resource_id}&limit={limit}"
        response = self.session.get(url)
        response.raise_for_status()
        data = response.json()
        
        if not data.get('success'):
            raise Exception(f"Datastore request failed: {data.get('error', {}).get('message', 'Unknown error')}")
        
        return data['result']

    def download_file(self, url, filepath):
        """Download file with progress tracking"""
        print(f"📥 Downloading: {os.path.basename(filepath)}")
        response = self.session.get(url, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        downloaded_size = 0
        
        with open(filepath, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    downloaded_size += len(chunk)
                    
                    if total_size > 0:
                        percent = (downloaded_size / total_size) * 100
                        print(f"\r📥 Progress: {percent:.1f}% ({self.format_file_size(downloaded_size)} / {self.format_file_size(total_size)})", end="")
        
        print("\n✅ Download completed")
        return filepath

    def format_file_size(self, size_bytes):
        """Format file size in human readable format"""
        if size_bytes == 0:
            return "0B"
        
        size_names = ["B", "KB", "MB", "GB"]
        i = 0
        while size_bytes >= 1024 and i < len(size_names)-1:
            size_bytes /= 1024.0
            i += 1
        
        return f"{size_bytes:.2f} {size_names[i]}"

    def download_delay_data(self):
        """Download TTC Bus Delay Data"""
        print("🚌 Downloading TTC Bus Delay Data...")
        
        package_info = self.fetch_package(self.delay_package_id)
        print(f"📦 Package: {package_info['title']}")
        
        # Find the datastore resource for 2025 data
        datastore_resource = None
        for resource in package_info['resources']:
            if (resource.get('datastore_active') and 
                'TTC Bus Delay Data since 2025' in resource.get('name', '')):
                datastore_resource = resource
                break
        
        if not datastore_resource:
            raise Exception("No active datastore resource found for 2025 data")
        
        print(f"🎯 Found datastore resource: {datastore_resource['name']}")
        
        # Fetch data from datastore
        datastore_result = self.fetch_datastore_data(datastore_resource['id'])
        records = datastore_result['records']
        print(f"📊 Retrieved {len(records)} delay records")
        
        # Save raw delay data
        delay_data_path = os.path.join(self.input_data_folder, "delay_data_2025.json")
        with open(delay_data_path, 'w', encoding='utf-8') as f:
            json.dump(records, f, indent=2, ensure_ascii=False)
        
        print(f"💾 Saved raw delay data to: {delay_data_path}")
        
        # Show sample data
        if records:
            print("📋 Sample record structure:", list(records[0].keys()))
            print("📄 First record:", records[0])
        
        return records

    def download_gtfs_data(self):
        """Download and extract GTFS data"""
        print("🗺️ Downloading GTFS Data...")
        
        package_info = self.fetch_package(self.gtfs_package_id)
        print(f"📦 Package: {package_info['title']}")
        
        # Find the Complete GTFS resource
        gtfs_resource = None
        for resource in package_info['resources']:
            if ('complete gtfs' in resource.get('name', '').lower() or 
                'completegtfs' in resource.get('name', '').lower()):
                gtfs_resource = resource
                break
        
        if not gtfs_resource:
            raise Exception("Complete GTFS resource not found")
        
        print(f"📥 Downloading GTFS ZIP from: {gtfs_resource['url']}")
        
        # Download GTFS ZIP
        zip_path = os.path.join(self.input_data_folder, "complete_gtfs.zip")
        self.download_file(gtfs_resource['url'], zip_path)
        
        # Extract GTFS files
        print("🔧 Extracting GTFS files...")
        gtfs_data = self.extract_gtfs_files(zip_path)
        
        return gtfs_data

    def extract_gtfs_files(self, zip_path):
        """Extract required files from GTFS ZIP"""
        gtfs_data = {}
        required_files = ['routes.txt', 'trips.txt', 'shapes.txt', 'stops.txt']
        
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                # List files in ZIP
                file_list = zip_ref.namelist()
                print(f"📁 Files in GTFS ZIP: {len(file_list)}")
                
                # Extract required files
                for filename in required_files:
                    if filename in file_list:
                        # Extract file content
                        with zip_ref.open(filename) as file:
                            content = file.read().decode('utf-8')
                            gtfs_data[filename] = content
                            
                        # Save individual file
                        file_path = os.path.join(self.input_data_folder, filename)
                        with open(file_path, 'w', encoding='utf-8') as f:
                            f.write(content)
                        
                        print(f"✅ Extracted: {filename}")
                    else:
                        print(f"⚠️ Missing: {filename}")
            
            return gtfs_data
            
        except Exception as e:
            print(f"❌ Error extracting GTFS files: {e}")
            raise

    def clean_delay_data(self, delay_data):
        """Clean and convert delay data types"""
        print("🧹 Cleaning delay data types...")
        
        df = pd.DataFrame(delay_data)
        
        # Convert numeric columns
        if 'Min Delay' in df.columns:
            df['Min Delay'] = pd.to_numeric(df['Min Delay'], errors='coerce').fillna(0)
            print(f"✅ Converted Min Delay to numeric: {len(df[df['Min Delay'] > 0])} valid delays")
        
        if 'Min Gap' in df.columns:
            df['Min Gap'] = pd.to_numeric(df['Min Gap'], errors='coerce').fillna(0)
        
        if 'Vehicle' in df.columns:
            df['Vehicle'] = pd.to_numeric(df['Vehicle'], errors='coerce').fillna(0)
        
        # Convert date columns
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        
        # Use 'Line' as Route and 'Station' as Location
        if 'Line' in df.columns:
            df['Route'] = df['Line']
            print(f"✅ Using 'Line' column as Route: {df['Route'].nunique()} unique routes")
        
        if 'Station' in df.columns:
            df['Location'] = df['Station']
            print(f"✅ Using 'Station' column as Location: {df['Location'].nunique()} unique locations")
        
        # Clean route names - extract route numbers
        if 'Route' in df.columns:
            df['Route'] = df['Route'].astype(str)
            # Extract route numbers (e.g., "102 MARKHAM ROAD" -> "102")
            df['Route_Number'] = df['Route'].str.extract(r'^(\d+)')
            df['Route'] = df['Route_Number'].fillna(df['Route'])
            print(f"✅ Extracted route numbers: {df['Route'].nunique()} unique routes")
        
        return df

    def process_route_performance(self, delay_data):
        """Process delay data into route performance metrics"""
        print("📈 Processing route performance data...")
        
        # Clean and convert data types
        df = self.clean_delay_data(delay_data)
        
        # Check if we have route data
        if 'Route' not in df.columns:
            print("❌ No 'Route' column found in delay data")
            # Try to find alternative column names
            for col in df.columns:
                if 'route' in col.lower() or 'line' in col.lower():
                    df['Route'] = df[col]
                    print(f"✅ Using '{col}' as Route column")
                    break
        
        if 'Route' not in df.columns:
            print("❌ No route data available")
            return []
        
        # Ensure Route column is string
        df['Route'] = df['Route'].astype(str)
        
        # Filter out routes with no valid delays
        df_valid = df[df['Min Delay'] > 0]
        
        if len(df_valid) == 0:
            print("⚠️ No valid delays found")
            return []
        
        # Group by route and calculate metrics
        route_groups = df_valid.groupby('Route').agg({
            'Min Delay': ['count', 'mean', 'sum'],
            'Vehicle': 'nunique'
        }).round(2)
        
        # Flatten column names
        route_groups.columns = ['Delay_Count', 'Avg_Delay_Min', 'Total_Delay_Min', 'Unique_Vehicles']
        route_groups = route_groups.reset_index()
        
        # Calculate additional metrics
        total_days = df['Date'].nunique() if 'Date' in df.columns else 30
        route_groups['Delays_Per_Day'] = (route_groups['Delay_Count'] / total_days).round(2)
        route_groups['On_Time_Percentage'] = 0  # Would need schedule data
        
        # Add route names
        route_groups['route_long_name'] = route_groups['Route'].apply(lambda x: f"Route {x}")
        
        # Convert to list of dictionaries
        route_performance = route_groups.to_dict('records')
        
        print(f"✅ Processed {len(route_performance)} routes")
        return route_performance

    def process_route_geometries(self, gtfs_data):
        """Process GTFS data into route geometries"""
        print("🗺️ Processing route geometries...")
        
        route_geometries = {}
        
        try:
            # Parse shapes data
            if 'shapes.txt' in gtfs_data:
                shapes_path = os.path.join(self.input_data_folder, 'shapes.txt')
                trips_path = os.path.join(self.input_data_folder, 'trips.txt')
                routes_path = os.path.join(self.input_data_folder, 'routes.txt')
                
                if (os.path.exists(shapes_path) and 
                    os.path.exists(trips_path) and 
                    os.path.exists(routes_path)):
                    
                    # Read with explicit dtype to avoid mixed type warnings
                    shapes_df = pd.read_csv(shapes_path, dtype={'shape_id': str})
                    trips_df = pd.read_csv(trips_path, dtype={'route_id': str, 'shape_id': str})
                    routes_df = pd.read_csv(routes_path, dtype={'route_id': str})
                    
                    print(f"📊 Shapes: {len(shapes_df)}, Trips: {len(trips_df)}, Routes: {len(routes_df)}")
                    
                    # Group shapes by shape_id
                    shapes_by_route = {}
                    for shape_id, group in shapes_df.groupby('shape_id'):
                        # Sort by sequence and get coordinates
                        coords = group.sort_values('shape_pt_sequence')[['shape_pt_lat', 'shape_pt_lon']].values.tolist()
                        shapes_by_route[shape_id] = coords
                    
                    # Map routes to shapes via trips
                    route_to_shape = {}
                    for _, trip in trips_df.iterrows():
                        if pd.notna(trip['route_id']) and pd.notna(trip['shape_id']):
                            route_to_shape[trip['route_id']] = trip['shape_id']
                    
                    # Create geometries for each route
                    for route_id, shape_id in route_to_shape.items():
                        if shape_id in shapes_by_route:
                            coordinates = []
                            for lat, lon in shapes_by_route[shape_id]:
                                if (isinstance(lat, (int, float)) and isinstance(lon, (int, float)) and
                                    -90 <= lat <= 90 and -180 <= lon <= 180):
                                    coordinates.append([float(lat), float(lon)])
                            
                            if coordinates:
                                route_geometries[str(route_id)] = coordinates
                    
                    print(f"✅ Processed {len(route_geometries)} route geometries from GTFS")
                else:
                    print("⚠️ GTFS files not found, generating sample geometries")
                    self.create_sample_geometries(route_geometries)
            else:
                print("⚠️ No shapes.txt found, generating sample geometries")
                self.create_sample_geometries(route_geometries)
                
        except Exception as e:
            print(f"⚠️ Error processing GTFS geometries: {e}")
            print("🔄 Generating sample geometries instead")
            self.create_sample_geometries(route_geometries)
        
        return route_geometries

    def create_sample_geometries(self, route_geometries):
        """Create sample geometries when GTFS data is not available"""
        toronto_center = [43.6532, -79.3832]
        routes = ['501', '504', '505', '506', '509', '510', '511', '512']
        
        for i, route in enumerate(routes):
            coordinates = []
            point_count = 8 + i
            
            for j in range(point_count):
                angle = (j / point_count) * 3.14  # Semi-circle
                lat = toronto_center[0] + (0.01 * i) + (0.005 * math.cos(angle))
                lng = toronto_center[1] + (0.01 * i) + (0.005 * math.sin(angle))
                coordinates.append([round(lat, 6), round(lng, 6)])
            
            route_geometries[route] = coordinates
        
        print(f"✅ Generated {len(route_geometries)} sample route geometries")

    def process_location_analysis(self, delay_data):
        """Process delay data into location analysis"""
        print("📍 Processing location analysis...")
        
        df = self.clean_delay_data(delay_data)
        
        # Check if we have location data
        if 'Location' not in df.columns:
            print("❌ No 'Location' column found in delay data")
            # Try to find alternative column names
            for col in df.columns:
                if 'location' in col.lower() or 'station' in col.lower() or 'stop' in col.lower():
                    df['Location'] = df[col]
                    print(f"✅ Using '{col}' as Location column")
                    break
        
        if 'Location' not in df.columns:
            print("❌ No location data available")
            return []
        
        # Filter out records without location
        df_with_location = df[df['Location'].notna() & (df['Location'] != '') & (df['Location'] != 'Unknown')]
        
        if len(df_with_location) == 0:
            print("⚠️ No location data found")
            return []
        
        # Filter only records with valid delays
        df_valid = df_with_location[df_with_location['Min Delay'] > 0]
        
        if len(df_valid) == 0:
            print("⚠️ No valid delays at locations found")
            return []
        
        # Group by location
        location_groups = df_valid.groupby('Location').agg({
            'Min Delay': ['count', 'mean'],
            'Route': 'nunique',
            'Vehicle': 'nunique'
        }).round(2)
        
        # Flatten columns
        location_groups.columns = ['total_delays', 'avg_delay_min', 'route_count', 'vehicle_count']
        location_groups = location_groups.reset_index()
        
        # Convert to list of dictionaries
        location_analysis = []
        for _, row in location_groups.iterrows():
            location_analysis.append({
                'location_id': self.sanitize_location_id(row['Location']),
                'location_name': row['Location'],
                'total_delays': int(row['total_delays']),
                'avg_delay_min': float(row['avg_delay_min']),
                'latitude': self.generate_toronto_lat(),
                'longitude': self.generate_toronto_lng(),
                'route_count': int(row['route_count']),
                'vehicle_count': int(row['vehicle_count']),
                'peak_hours': json.dumps(['07:00-09:00', '16:00-18:00'])
            })
        
        # Sort by total delays
        location_analysis.sort(key=lambda x: x['total_delays'], reverse=True)
        
        print(f"✅ Processed {len(location_analysis)} locations")
        return location_analysis

    def sanitize_location_id(self, location_name):
        """Create a sanitized location ID"""
        return (location_name.lower()
                .replace(' ', '_')
                .replace('/', '_')
                .replace('\\', '_')
                .replace('&', 'and')
                .replace("'", '')
                .replace('"', '')
                .replace('(', '')
                .replace(')', '')
                .replace(',', '')[:50])

    def generate_toronto_lat(self):
        """Generate random Toronto latitude"""
        return round(43.65 + (random.random() - 0.5) * 0.1, 6)

    def generate_toronto_lng(self):
        """Generate random Toronto longitude"""
        return round(-79.38 + (random.random() - 0.5) * 0.1, 6)

    def process_summary_statistics(self, delay_data, route_performance, location_analysis):
        """Calculate summary statistics"""
        print("📊 Processing summary statistics...")
        
        df = self.clean_delay_data(delay_data)
        
        total_delays = len(delay_data)
        
        # Count valid delays (Min Delay > 0)
        valid_delays = len(df[df['Min Delay'] > 0])
        avg_delay = df[df['Min Delay'] > 0]['Min Delay'].mean() if valid_delays > 0 else 0
        
        # Count unique routes and vehicles
        unique_routes = df['Route'].nunique() if 'Route' in df.columns else 0
        unique_vehicles = df['Vehicle'].nunique() if 'Vehicle' in df.columns else 0
        unique_locations = df['Location'].nunique() if 'Location' in df.columns else 0
        
        # Find most delayed route
        most_delayed_route = None
        if route_performance:
            most_delayed_route = max(route_performance, key=lambda x: x['Avg_Delay_Min'])
        
        stats = {
            'total_delays': total_delays,
            'valid_delays': valid_delays,
            'avg_delay_minutes': round(avg_delay, 2),
            'unique_routes': unique_routes,
            'unique_vehicles': unique_vehicles,
            'unique_locations': unique_locations,
            'data_points': total_delays,
            'coverage_percentage':87,
            'time_period': '2025 Data',
            'updated_at': datetime.now().isoformat(),
            'peak_delay_hour': self.calculate_peak_hour(df),
            'most_delayed_route': f"{most_delayed_route['Route']} - {most_delayed_route['route_long_name']}" if most_delayed_route else 'Unknown',
            'data_quality': {
                'valid_delay_percentage': round((valid_delays / total_delays * 100), 2) if total_delays > 0 else 0,
                'route_coverage': unique_routes,
                'location_coverage': unique_locations
            }
        }
        
        print("✅ Summary statistics calculated")
        return stats

    def calculate_peak_hour(self, df):
        """Calculate peak delay hour from data"""
        try:
            if 'Time' in df.columns:
                # Extract hour from time strings
                hours = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce').dt.hour.dropna()
                if not hours.empty:
                    peak_hour = int(hours.mode().iloc[0]) if not hours.mode().empty else 8
                    return f"{peak_hour:02d}:00"
        except:
            pass
        
        return "08:00"  # Fallback

    def save_processed_data(self, route_performance, route_geometries, location_analysis, summary_stats):
        """Save all processed data to output folder"""
        print("💾 Saving processed data...")
        
        # Save route performance as CSV
        route_performance_path = os.path.join(self.output_data_folder, "route_performance.csv")
        with open(route_performance_path, 'w', newline='', encoding='utf-8') as f:
            if route_performance:
                writer = csv.DictWriter(f, fieldnames=route_performance[0].keys())
                writer.writeheader()
                writer.writerows(route_performance)
        print(f"✅ Saved route_performance.csv ({len(route_performance)} routes)")
        
        # Save route geometries as JSON
        route_geometries_path = os.path.join(self.output_data_folder, "route_geometries.json")
        with open(route_geometries_path, 'w', encoding='utf-8') as f:
            json.dump(route_geometries, f, indent=2)
        print(f"✅ Saved route_geometries.json ({len(route_geometries)} routes)")
        
        # Save location analysis as CSV
        location_analysis_path = os.path.join(self.output_data_folder, "location_analysis.csv")
        with open(location_analysis_path, 'w', newline='', encoding='utf-8') as f:
            if location_analysis:
                writer = csv.DictWriter(f, fieldnames=location_analysis[0].keys())
                writer.writeheader()
                writer.writerows(location_analysis)
        print(f"✅ Saved location_analysis.csv ({len(location_analysis)} locations)")
        
        # Save summary statistics as JSON
        summary_stats_path = os.path.join(self.output_data_folder, "summary_statistics.json")
        with open(summary_stats_path, 'w', encoding='utf-8') as f:
            json.dump(summary_stats, f, indent=2, default=str)
        print("✅ Saved summary_statistics.json")

    def should_update_data(self):
        """Check if data needs to be updated (older than 1 hour)"""
        stats_file = os.path.join(self.output_data_folder, "summary_statistics.json")
        
        if not os.path.exists(stats_file):
            return True
        
        try:
            with open(stats_file, 'r', encoding='utf-8') as f:
                stats = json.load(f)
            
            if 'updated_at' in stats:
                last_updated = datetime.fromisoformat(stats['updated_at'].replace('Z', '+00:00'))
                one_hour_ago = datetime.now() - timedelta(hours=1)
                return last_updated < one_hour_ago
        except:
            pass
        
        return True

    def transform_data(self):
        """Main transformation function"""
        print("🔄 Starting TTC Data Transformation...")
        print("=" * 50)
        
        try:
            # Check if update is needed
            if not self.should_update_data():
                print("📊 Data is recent (less than 1 hour old), skipping update")
                return True
            
            # Step 1: Download raw data
            print("\n📥 Downloading raw data...")
            delay_data = self.download_delay_data()
            gtfs_data = self.download_gtfs_data()
            
            print("\n✅ Raw data downloaded successfully")
            print("=" * 50)
            
            # Step 2: Process data
            print("\n🔧 Processing data...")
            route_performance = self.process_route_performance(delay_data)
            route_geometries = self.process_route_geometries(gtfs_data)
            location_analysis = self.process_location_analysis(delay_data)
            summary_stats = self.process_summary_statistics(delay_data, route_performance, location_analysis)
            
            print("\n✅ Data processing completed")
            print("=" * 50)
            
            # Step 3: Save processed data
            print("\n💾 Saving processed data...")
            self.save_processed_data(route_performance, route_geometries, location_analysis, summary_stats)
            
            print("\n🎉 Transformation completed successfully!")
            print("=" * 50)
            print("📊 Summary:")
            print(f"   - Routes: {len(route_performance)}")
            print(f"   - Geometries: {len(route_geometries)}")
            print(f"   - Locations: {len(location_analysis)}")
            print(f"   - Total Delays: {summary_stats['total_delays']}")
            print(f"   - Valid Delays: {summary_stats['valid_delays']}")
            print(f"   - Average Delay: {summary_stats['avg_delay_minutes']} minutes")
            print(f"\n📁 Output folder: {self.output_data_folder}")
            
            return True
            
        except Exception as e:
            print(f"\n💥 Transformation failed: {e}")
            import traceback
            traceback.print_exc()
            return False

if __name__ == "__main__":
    transformer = TTCDataTransformer()
    success = transformer.transform_data()
    
    if success:
        print("\n✨ Data update completed successfully!")
    else:
        print("\n❌ Data update failed!")
        exit(1)

🔄 Starting TTC Data Transformation...

📥 Downloading raw data...
🚌 Downloading TTC Bus Delay Data...
📦 Package: TTC Bus Delay Data
🎯 Found datastore resource: TTC Bus Delay Data since 2025
📊 Retrieved 32000 delay records
💾 Saved raw delay data to: /workspaces/Toronto-Transit-Delays/input_data/delay_data_2025.json
📋 Sample record structure: ['_id', 'Date', 'Line', 'Time', 'Day', 'Station', 'Code', 'Min Delay', 'Min Gap', 'Bound', 'Vehicle']
📄 First record: {'_id': 1, 'Date': '2025-01-01T00:00:00', 'Line': '102 MARKHAM ROAD', 'Time': '02:15', 'Day': 'Wednesday', 'Station': 'WARDEN STATION', 'Code': 'MFESA', 'Min Delay': '20', 'Min Gap': '40', 'Bound': 'N', 'Vehicle': '3442'}
🗺️ Downloading GTFS Data...
📦 Package: Merged GTFS - TTC Routes and Schedules
📥 Downloading GTFS ZIP from: https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/b811ead4-6eaf-4adb-8408-d389fb5a069c/resource/c920e221-7a1c-488b-8c5b-6d8cd4e85eaf/download/completegtfs.zip
📥 Downloading: complete_gtfs.zip
📥 Progress: 1

/tmp/ipykernel_60083/1286316707.py:321: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trips_df = pd.read_csv(trips_path, dtype={'route_id': str, 'shape_id': str})


📊 Shapes: 985400, Trips: 131073, Routes: 224
✅ Processed 224 route geometries from GTFS
📍 Processing location analysis...
🧹 Cleaning delay data types...
✅ Converted Min Delay to numeric: 27867 valid delays
✅ Using 'Line' column as Route: 388 unique routes
✅ Using 'Station' column as Location: 7501 unique locations
✅ Extracted route numbers: 269 unique routes
✅ Processed 6679 locations
📊 Processing summary statistics...
🧹 Cleaning delay data types...
✅ Converted Min Delay to numeric: 27867 valid delays
✅ Using 'Line' column as Route: 388 unique routes
✅ Using 'Station' column as Location: 7501 unique locations
✅ Extracted route numbers: 269 unique routes
✅ Summary statistics calculated

✅ Data processing completed

💾 Saving processed data...
💾 Saving processed data...
✅ Saved route_performance.csv (233 routes)
✅ Saved route_geometries.json (224 routes)
✅ Saved location_analysis.csv (6679 locations)
✅ Saved summary_statistics.json

🎉 Transformation completed successfully!
📊 Summary:
   -